[![img](img/pythonista.png)](https://www.pythonista.io)

# Desarrollo de una aplicación web simple.

## Objetivos.

1. Desplegar un documento HTML con los datos completos del objeto de tipo *dict* contenido en la representación del objeto tipo *list* guardado en [data/alumnos.txt](data/alumnos.txt), cuando el valor correspondiente al identificador *'Cuenta'* coincida con el número ingresado como parte de la URL ```http://localhost:5000/alumno/<número>```. 

* Desplegar un documento HTML que contenga todas las coincidencias de la búsqueda de la cadena que se ingrese como parte de la URL ```http://localhost:5000/busca/<cadena>``` en los objetos de tipo *dict* contenidos en la representación del objeto tipo *list* guardado en [data/alumnos.txt](data/alumnos.txt).
    * La busqueda se realizará en los valores correspondientes a los identificadores *'Nombre'*, *'Primer Apellido'* y *'Segundo Apellido'*.
    * El documento HTML mostrará la lista de coincidencias de los objetos tipo _dict_ incluyendo los valores correspondientes a *'Nombre'*, *'Primer Apellido'*, *'Segundo Apellido'*, así como una URL que incluya el número correspondiente a *Cuenta* en el formato ```http://localhost:5000/alumno/<número>```.

## Plantillas.

Los documentos HTML se elaborarán a partir de platillas de Jinja 2.

### Plantilla para ```http://localhost:5000/alumno/<número>```.

La plantilla [templates/despliega.html](templates/despliega.html) contiene el siguiente código:

```html
<h1> Alumno {{ alumno['Cuenta'] }} </h1>
<ul>
<li>Nombre: {%for campo in ['Nombre', 'Primer Apellido', 'Segundo Apellido'] %}
{{alumno[campo]}}{% endfor %}</li>
<li>Carrera: {{ alumno['Carrera'] }} </li>
<li>Semestre: {{ alumno['Semestre'] }} </li>
<li>Promedio: {{ alumno['Promedio'] }} </li>
{% if alumno["Al Corriente"] %} <li>El alumno está al corriente de pagos.</li> {% endif %}
</ul>
```

### Plantilla para ```http://localhost:5000/busca/<cadena>```.

La plantilla [templates/busqueda_avanzada.html](templates/busqueda_avanzada.html) contiene el siguiente código:

``` html
<h1> Alumnos Encontrados</h1>
<ul>
{%for alumno in alumnos %}
<li> <a href={{ url_for('despliega', cuenta=alumno['Cuenta']) }}> {{ alumno['Cuenta']}}</a>: 
{%for campo in ['Nombre', 'Primer Apellido', 'Segundo Apellido'] %}
{{alumno[campo]}}
{% endfor %} </li> 
{% endfor %}
</ul>
```

## Código de la aplicación.

### Sección de datos.

In [ ]:
campos = ('Nombre', 'Primer Apellido', 'Segundo Apellido')
ruta = 'data/alumnos.txt'

### La función ```encuentra()``` .

* Busca una cadena de caracteres dentro de los campos indicados de un objeto tipo ```dict```. 
* En caso de encontrar una coincidencia, el resultado es ```True```.

In [ ]:
encuentra = lambda cadena, registro, campos: bool(sum([cadena.casefold() \
            in registro[campo].casefold() for campo in campos]))

### La función ```buscar_archivo()```.

* Lee el contenido del archivo de texto indicado en el parámetro ```ruta``` y lo transforma mediante la función ```eval()```. Se da por sentado de que el objeto `` `base``` es de tipo ```tuple``` o ```list```que a su vez contiene objetos tipo ```dict```.
* A cada elemento del objeto ```base``` se le aplica la función ```encuentra()``` y se crea una lista de aquellos elementos en los que exista una coicidencia de la cadena en los campos indicados.

In [ ]:
def buscar_archivo(cadena, ruta, campos):
    with open(ruta, 'tr') as archivo:
        base = eval(archivo.read())
    return [registro for registro in base if encuentra(cadena, registro, campos)]

Se importan los componentes requeridos.

In [ ]:
import jinja2
from flask import Flask, render_template, url_for, abort

Se instancia el objeto *app* a partir de la clase *Flask*.

In [ ]:
app = Flask(__name__)

Se crea la función de vista para ```http://localhost:5000/busca/<cadena>```.

In [ ]:
@app.route('/busca/<cadena>')
def busca(cadena):
    return render_template('busqueda_avanzada.html', alumnos=buscar_archivo(str(cadena), ruta, campos))

Se crea la función de vista para ```http://localhost:5000/alumno/<cuenta>```.

In [ ]:
@app.route('/alumno/<cuenta>')
def despliega(cuenta):
    falla = True
    with open(ruta, 'tr') as archivo:
        base = eval(archivo.read())
    for registro in base:
        try: 
            if registro['Cuenta'] == int(cuenta):
                alumno = registro
                falla = False
                break
        except:
            pass
    if falla :
        abort(404)
                
    return render_template('despliega.html', alumno=alumno)

Se crea la función de vista en caso de un error 404.

In [ ]:
@app.errorhandler(404)
def no_encontrado(error):
    return '<h1> Error</h1><p>Recurso no encontrado.</p>', 404

**Advertencia:** Una vez ejecutada la siguente celda es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

**Ejemplos:**

* Regresa la lista de coincidencias usando la cadena *Ramos*.
    * http://localhost:5000/busca/Ramos 

* Regresa al registro con el campo 'Cuenta' igual a *1231223*.
    * http://localhost:5000/alumno/1231223
    
* Regresa la página de error 404.
    * http://localhost:5000/alumno/1231217

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>